<a href="https://colab.research.google.com/github/valerianXZ/Project/blob/main/WebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pygsheets selenium
import pandas as pd
import requests
import pygsheets
import json
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
!apt-get install -y chromium-browser
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd
import pygsheets

In [ ]:
from google.colab import files
uploaded=files.upload()

In [24]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
)


service = Service(executable_path="/usr/lib/chromium-browser/chromedriver")
driver = webdriver.Chrome( options=options)


In [25]:
# Get data from Google Search Engine
query = "Machine learning skills needed 2025"
search_url = f"https://www.google.com/search?q={query}&hl=zh-TW&gl=tw"

print("Opening Google Search...")
driver.get(search_url)
time.sleep(5)

urls=[]
search_results = driver.find_elements(By.CSS_SELECTOR, "div.tF2Cxc")

print(f"Found {len(search_results)} search results")


for result in search_results:
    try:
        title = result.find_element(By.TAG_NAME, "h3").text
        link_element = result.find_element(By.TAG_NAME, "a")
        link = link_element.get_attribute("href")

        print(f"Title: {title}, URL: {link}")
        urls.append((title, link))
    except Exception as e:
        print(f"Error extracting title/link: {e}")

# If no URLs were found, stop execution
if not urls:
    print("No search results found. Exiting...")
    driver.quit()
    exit()

data = []
for title, url in urls:
    try:
        print(f"Visiting: {url}")
        driver.get(url)
        time.sleep(5)


        try:
            meta_description = driver.find_element(By.NAME, "description").get_attribute("content")
        except:
            meta_description = "No meta description found"

        # Get the  Content
        paragraphs = driver.find_elements(By.TAG_NAME, "p")
        content = "\n".join([p.text for p in paragraphs])  # Limit to first 10 paragraphs for faster processing

        data.append([title, meta_description, content, url])
    except Exception as e:
        print(f"Error extracting content from {url}: {e}")

driver.quit()

Opening Google Search...
Found 14 search results
Title: 12 Machine Learning Skills to Power Your Career in 2025, URL: https://careerfoundry.com/en/blog/data-analytics/machine-learning-skills/
Title: The ML Engineer: 7 Essential Skills to Master in 2025, URL: https://medium.com/@piyushkashyap045/the-ml-engineer-7-essential-skills-to-master-in-2025-25135d48640f
Title: Machine Learning Engineer Skills, URL: https://www.tealhq.com/skills/machine-learning-engineer
Title: Future-Proof Your Machine Learning Career in 2025, URL: https://machinelearningmastery.com/future-proof-machine-learning-career-2025/
Title: The 2025 Machine Learning Job Market: Trends, Salaries, ..., URL: https://www.linkedin.com/pulse/2025-machine-learning-job-market-trends-salaries-skills-shields-16bye
Title: AI & Machine Learning Careers in 2025: Skills and Growth ..., URL: https://medium.com/@vaishnaviyada/ai-machine-learning-careers-in-2025-skills-and-growth-opportunities-fc284c1bf09d
Title: Must-Have Machine Learnin

In [33]:
#Set as dataFrame and import to google sheet
from tabulate import tabulate
df = pd.DataFrame(data, columns=["Title", "Meta_Description", "Content", "URL"])
df["Content"]=df["Content"].str.replace(r'\n+', ' ',regex=True)
print("Scraping complete. Data preview:")
print(tabulate(df.head(10),headers='keys',tablefmt='github'))

gc = pygsheets.authorize(service_file="practice-449103-1a3eb945fca7.json")
sh = gc.open("DataSkills")
wks = sh.sheet1
wks.set_dataframe(df, (1, 1))

print("Data successfully uploaded to Google Sheets!")


Scraping complete. Data preview:
|    | Title                                                        | Meta_Description                                                                                                                                                                                                                                                                                             | Content                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           